<h1>Roll Balance and Chassis Equations


Sources:
RCVD...

In [51]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from IPython.display import display, Markdown
import sympy


disp_md = lambda text: display(Markdown(text))


<h3>Initial Ride Rate Calculations</h3>

Non ground effect Indy Cars ran 95-120cpm(1.58-2.0000Hz ) which seems like a good place to start - RCVD 16.2

To insure initial understeer, calculate the Total Lateral
Load Transfer Distribution (TLLTD) to be 5% more than
the weight distribution at the front - RCVD table 16.5

In [ ]:
vehicle_mass = 250 + 80 #kg
rear_mass_ratio = 0.6

rear_mass = vehicle_mass*rear_mass_ratio
front_mass = vehicle_mass*(1-rear_mass_ratio)

def ride_rate(natural_frequency, mass):
    return mass * natural_frequency**2


target_frequency = 2.0 * 2*np.pi # rad/s
rear_wheel_ride_rate = ride_rate(target_frequency, rear_mass/2)
disp_md(f'The ideal individual rear wheel ride rate with an undamped natural frequency of {target_frequency /(np.pi*2)} Hz is **{rear_wheel_ride_rate*1e-3 :.2f} N/mm**; **{5.7101471627692*rear_wheel_ride_rate*1e-3 :.2f} lb/in**')



front_wheel_ride_rate = ride_rate(target_frequency, front_mass/2)
disp_md(f'The ideal individual front wheel ride rate for undamped natural frequency of {target_frequency /(np.pi*2)} Hz is **{front_wheel_ride_rate*1e-3 :.2f} N/mm**; **{5.7101471627692*front_wheel_ride_rate*1e-3 :.2f} lb/in**')


front_natural_frequency = (rear_wheel_ride_rate/(front_mass/2))**(1/2)
disp_md(f'The front natural frequency using the same spring rates as the rear is **{front_natural_frequency/(2*np.pi) :.2f} hz**')


The ideal individual rear wheel ride rate with an undamped natural frequency of 2.0 Hz is **15.63 N/mm**; **89.27 lb/in**

The ideal individual front wheel ride rate for undamped natural frequency of 2.0 Hz is **10.42 N/mm**; **59.51 lb/in**

The front natural frequency using the same spring rates as the rear is **2.45 hz**

<h3>Instant Rear Roll Center Placement</h3> 

In [ ]:
def roll_center_height(desired_wheel_deflection, track, wheel_rate, cg_height, axle_mass, lateral_acceleration):
    return cg_height - desired_wheel_deflection*track*wheel_rate/(2*axle_mass*lateral_acceleration)

total_wheel_travel = 35e-3 #m
roll_travel_ratio = 0.8

track_width = 1300e-3 #m
wheelbase = 1530e-3 #m

center_of_mass_height = 220*1e-3 + total_wheel_travel #m
g = 9.81 #m/s^2
maximum_lateral_acceleration = 1.3*g#m/s^2

roll_axis_inclination = 2 #degrees

roll_wheel_deflection = total_wheel_travel*(0.8)
rear_roll_center_height = roll_center_height(roll_wheel_deflection, track_width, rear_wheel_ride_rate, center_of_mass_height, vehicle_mass/2, maximum_lateral_acceleration)
disp_md(f'The rear roll center height needed to meet target rear wheel travel of {total_wheel_travel*roll_travel_ratio*1e+3 :.2f} mm in roll is **{rear_roll_center_height*1e+3 :.2f} mm**')



front_roll_center_height = rear_roll_center_height - np.tan(np.radians(roll_axis_inclination))*wheelbase
disp_md(f'The front roll center height needed to give a roll axis inclination of {roll_axis_inclination} degrees is **{front_roll_center_height*1e+3 :.2f} mm**')


maximum_body_roll_angle = np.degrees(np.atan(2*roll_wheel_deflection)/track_width)
disp_md(f'\nThe body roll angle at a lateral acceleration of {maximum_lateral_acceleration :.2f} m/s^2 is **{maximum_body_roll_angle :.2f} degrees**')

rear_roll_gradient = maximum_body_roll_angle / maximum_lateral_acceleration  * g
disp_md(f'The rear roll gradient from the chosen parameters is **{rear_roll_gradient :.2f} degrees/g**')


The roll center height needed to meet target rear wheel travel of 28.00 mm in roll is **119.78 mm**

The front roll center height needed to give a roll axis inclination of 2 degrees is **66.35 mm**


The body roll angle at a lateral acceleration of 12.75 m/s^2 is **2.47 degrees**

The rear roll gradient from the chose parameters is **1.90 degrees/g**

<h2> Symbolically set up load transfer equations

In [ ]:
total_load_transfer_symb, load_transfer_front_symb, load_transfer_rear_symb  = sympy.symbols('LT LT_f LT_r')
roll_stiffness_front_symb, roll_stiffness_rear_symb, chassis_stiffness_symb, lateral_acceleration_symb = sympy.symbols('K_f K_r K_ch a_c')
track_symb, center_of_mass_height_symb, RCH_f_symb, RCH_r_symb = sympy.symbols('T H_cg RCH_f RCH_r')
mass_symb, mass_front_symb, mass_rear_symb = sympy.symbols('m m_f m_r')

phi_1_symb, phi_2_symb, front_inertial_moment_symb, rear_inertial_moment_symb, avg_wheel_travel_symb = sympy.symbols('phi_1 phi_2 M_f M_r ΔW')
f_lt_ratio_symb, f_roll_spring_ratio_symb, r_mass_ratio_symb, chassis_tuning_constant_symb, lt_to_acceleration_ratio_symb, roll_center_diff_symb = sympy.symbols('η λ γ κ ξ δ')


front_moment_balance = sympy.Eq(front_inertial_moment_symb, roll_stiffness_front_symb*phi_1_symb - chassis_stiffness_symb*(phi_2_symb-phi_1_symb))
rear_moment_balance = sympy.Eq(rear_inertial_moment_symb, roll_stiffness_rear_symb*phi_2_symb + chassis_stiffness_symb*(phi_2_symb-phi_1_symb))

disp_md('## Moment Balance Equations:')
display(front_moment_balance)
display(rear_moment_balance)

chassis_tuning_eq = sympy.Eq(chassis_tuning_constant_symb**-1, (f_roll_spring_ratio_symb-1)*(f_lt_ratio_symb+1)/((f_lt_ratio_symb-1)*(f_roll_spring_ratio_symb+1))) #This inverse doesnt match my notes but it seems to give better results
disp_md('## Chassis Tuning Constant Equation:')
display(chassis_tuning_eq)

wheel_travel_equation = sympy.Eq(avg_wheel_travel_symb, track_symb/2 * (phi_1_symb+phi_2_symb)/2)
disp_md('## Average Wheel Travel In Roll Equation:')
display(wheel_travel_equation)

roll_angles = sympy.solve([front_moment_balance, rear_moment_balance], [phi_1_symb, phi_2_symb])
load_transfer_equations = [sympy.Eq(roll_stiffness_front_symb*roll_angles[phi_1_symb]/track_symb, load_transfer_front_symb), 
                           sympy.Eq(roll_stiffness_rear_symb*roll_angles[phi_2_symb]/track_symb, load_transfer_rear_symb), 
                           chassis_tuning_eq,
                           wheel_travel_equation.subs(roll_angles)]

disp_md('## Load Transfer System of Equations:')
display(*load_transfer_equations)


#total_load_transfer = 2*mass*ac*h/t

vals = [(roll_stiffness_front_symb, f_roll_spring_ratio_symb*roll_stiffness_rear_symb), 
        
        (front_inertial_moment_symb, mass_front_symb*lateral_acceleration_symb*(center_of_mass_height_symb-RCH_f_symb)), 
        (rear_inertial_moment_symb, mass_rear_symb*lateral_acceleration_symb*(center_of_mass_height_symb-RCH_r_symb)), 


        (load_transfer_front_symb, load_transfer_rear_symb* f_lt_ratio_symb), 
        (load_transfer_rear_symb, total_load_transfer_symb/(1+f_lt_ratio_symb)),

        (mass_front_symb, (1-r_mass_ratio_symb)*mass_symb), 
        (mass_rear_symb, r_mass_ratio_symb*mass_symb),

        (total_load_transfer_symb, lt_to_acceleration_ratio_symb*lateral_acceleration_symb),


        (RCH_f_symb, RCH_r_symb-roll_center_diff_symb)
        ]

nondim_equations = [expr.subs(vals).simplify() for expr in load_transfer_equations]

disp_md('## Nondim Load Transfer Equations:')
display(*nondim_equations)


## Moment Balance Equations:

Eq(M_f, -K_ch*(-phi_1 + phi_2) + K_f*phi_1)

Eq(M_r, K_ch*(-phi_1 + phi_2) + K_r*phi_2)

## Chassis Tuning Constant Equation:

Eq(1/κ, (η + 1)*(λ - 1)/((η - 1)*(λ + 1)))

...

## Nondim Load Transfer Equations:

Eq(a_c*η*ξ/(η + 1), -a_c*m*λ*(-K_ch*γ*(H_cg - RCH_r) + K_ch*(γ - 1)*(H_cg - RCH_r + 0.05) + K_r*(γ - 1)*(H_cg - RCH_r + 0.05))/(T*(K_ch*λ + K_ch + K_r*λ)))

Eq(a_c*ξ/(η + 1), a_c*m*(K_ch*γ*(H_cg - RCH_r) - K_ch*(γ - 1)*(H_cg - RCH_r + 0.05) + K_r*γ*λ*(H_cg - RCH_r))/(T*(K_ch*λ + K_ch + K_r*λ)))

Eq(1/κ, (η + 1)*(λ - 1)/((η - 1)*(λ + 1)))

Eq(ΔW, T*a_c*m*(2*K_ch*γ*(H_cg - RCH_r) - 2*K_ch*(γ - 1)*(H_cg - RCH_r + 0.05) + K_r*γ*λ*(H_cg - RCH_r) - K_r*(γ - 1)*(H_cg - RCH_r + 0.05))/(4*K_r*(K_ch*λ + K_ch + K_r*λ)))

<h3> Front ARB and chassis stiffness </h3>

In [99]:
rear_spring_roll_stiffness = track_width**2 * rear_wheel_ride_rate / 4 
disp_md(f'The rear roll rate is **{rear_spring_roll_stiffness*2*np.pi/180 :.2f} Nm/deg**')

front_load_transfer_ratio = 1.05 #slightly higher front load transfer is to give margin for understeer
chassis_tuning_constant = 0.85  #ratio of how a change in roll stiffness balance affects the load transfer balance

variable_values = [
    (track_symb, track_width), 
    (center_of_mass_height_symb, center_of_mass_height), 
    #(RCH_f_symb, front_roll_center_height), 
    #(RCH_r_symb, rear_roll_center_height), 
    (mass_symb, vehicle_mass),
    (f_lt_ratio_symb, front_load_transfer_ratio),
    (r_mass_ratio_symb, rear_mass_ratio), 
    (roll_stiffness_rear_symb, rear_spring_roll_stiffness),
    (chassis_tuning_constant_symb, chassis_tuning_constant),  #ratio of how a change in roll stiffness balance affects the load transfer balance
    (lateral_acceleration_symb, 1.3*9.81),
    (avg_wheel_travel_symb, total_wheel_travel*roll_travel_ratio)
]

disp_md('## Vehicle Parameters:')
table = "| Parameter | Value |\n|---|---|\n" + "".join([f"| {eq[0]} | {eq[1] : .3f} |\n" for eq in variable_values])
disp_md(table)

numeric_equations = [eq.subs(variable_values).simplify() for eq in nondim_equations]
disp_md('\n ## Final System of Equations:')
display(*numeric_equations)  
chassis_stiffness, front_roll_stiffness_ratio, lt_to_ac_ratio, rear_roll_center_height  = sympy.solve(numeric_equations, [chassis_stiffness_symb, f_roll_spring_ratio_symb, lt_to_acceleration_ratio_symb, RCH_r_symb])[0]


disp_md(f'Chassis Stiffness: **{chassis_stiffness*np.pi/180 :.2f} Nm/deg**')
disp_md(f'Front spring roll stiffness ratio: **{front_roll_stiffness_ratio :.4f}**')
disp_md(f'LT/ac: **{lt_to_ac_ratio :.2f}**')
disp_md(f'The Rear Roll Center Height is **{rear_roll_center_height*1e+3 :.2f} mm**')

front_roll_spring_rate = front_roll_stiffness_ratio*rear_spring_roll_stiffness
disp_md(f'\nThe front roll rate which give a front load transfer ratio of {chassis_tuning_constant} is **{front_roll_spring_rate*2*np.pi/180 :.2f} Nm/deg**')

front_roll_rate_from_ride = track_width**2 * front_wheel_ride_rate / 4 #TODO double check constants in equation
front_arb_contribution = (front_roll_spring_rate-front_roll_rate_from_ride)/front_roll_spring_rate
disp_md(f'The front ARB contributes **{front_arb_contribution*100 :.2f}%** to the front roll stiffness')


The rear roll rate is **230.56 Nm/deg**

## Vehicle Parameters:

| Parameter | Value |
|---|---|
| T |  1.300 |
| H_cg |  0.255 |
| m |  330.000 |
| η |  1.050 |
| γ |  0.600 |
| K_r |  6605.134 |
| κ |  0.850 |
| a_c |  12.753 |
| W |  0.028 |



 ## Final System of Equations:

Eq(6.5320243902439*ξ, λ*(-3237.3*K_ch*RCH_r + 890.2575*K_ch - 8553120.18322968*RCH_r + 2608701.65588505)/(1.0*K_ch*λ + 1.0*K_ch + 6605.13404938504*λ))

...

Front spring roll stiffness ratio: **1.0591**

LT/ac: **35.33**

The Rear Roll Center Height is **135.81 mm**


The front roll rate which give a front load transfer ratio of 0.85 is **244.19 Nm/deg**

The front ARB contributes **37.05%** to the front roll stiffness

<h3>Suspension Pickup Point Calculation</h3>



In [ ]:
loaded_wheel_radius = 230e-3 #m
upright_vertical_spacing = 250e-3 #m
minimum_lca_centerline_distance = 150e-3 #m         This is the minimum distance from the LCA chassis pickup to the centerline in order to adhere to rules

upright_pickup_offset = 50e-3 #m
kpi = np.radians(5) #rads

dynamic_camber = np.radians(8) #rads        This is a wild guess at camber to give maximum lateral acceleration


